In [1]:
#pip install -r requirements.txt

# Pacote

In [1]:
import pandas as pd
import chardet
import glob
from unidecode import unidecode
import os
import Levenshtein as lev
import csv 

pd.set_option('display.max_columns', None)

# Funções

In [2]:
def detect_encoding(file_pattern_or_path, num_bytes=10000):
    """
    Detecta a codificação do arquivo ou arquivos fornecidos.
    
    Parâmetros:
        file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
        num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
    
    Retorna:
        dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
    """
    
    # Encontrar arquivos que correspondem ao padrão ou caminho fornecido.
    files = glob.glob(file_pattern_or_path)
    encodings = {}

    # Iterar sobre cada arquivo encontrado.
    for file_path in files:
        # Abrir arquivo em modo binário e ler os primeiros 'num_bytes' bytes.
        with open(file_path, 'rb') as f:
            rawdata = f.read(num_bytes)
            # Detectar a codificação do fragmento lido e armazenar no dicionário.
            encodings[file_path] = chardet.detect(rawdata)["encoding"]
    
    return encodings

def is_delayed(row):
    """
    Determina se um voo está atrasado ou pontual com base na diferença de tempo entre a partida prevista e a real.

    Parâmetros:
    row (pd.Series): Uma linha de um DataFrame que contém informações de partida do voo.

    Retorna:
    str: "Atrasado" se o voo estiver atrasado por mais de 15 minutos, caso contrário "Pontual".
    """
    # Calcula a diferença de tempo entre a partida real e a prevista.
    delta = row['Partida Real'] - row['Partida Prevista']
    # Retorna "Atrasado" se a diferença for maior que 15 minutos, senão "Pontual".
    return "Atrasado" if delta > pd.Timedelta(minutes=15) else "Pontual"

def padronizar_nome_coluna(coluna):
    """
    Padroniza o nome de uma coluna removendo acentos, transformando em minúsculas e substituindo espaços, hífens, 
    parênteses, pontos, vírgulas, porcentagens e barras por underscores ou removendo-os. Remove underscores duplicados.

    Parâmetros:
    coluna (str): Nome da coluna a ser padronizado.

    Retorna:
    str: Nome da coluna padronizado.
    """
    from unidecode import unidecode

    # Remover acentos do nome da coluna.
    coluna = unidecode(coluna)
    # Transformar todas as letras em minúsculas.
    coluna = coluna.lower()
    # Substituir espaços, hífens, parênteses, pontos, vírgulas, porcentagens, barras e barras invertidas por underscores ou removendo-os.
    coluna = coluna.replace(' ', '_').replace('-', '_').replace('(', '_').replace(')', '_').replace('.', '_').replace(',', '_').replace('%', 'pcnt').replace('/', '').replace('\\', '')
    # Substituir underscores duplicados por um único underscore.
    while '__' in coluna:
        coluna = coluna.replace('__', '_')

    return coluna

def calculate_time_delta(row, start_col, end_col):
    """
    Calcula a diferença de tempo entre duas colunas de um DataFrame.

    Parâmetros:
    row (pd.Series): Uma linha do DataFrame.
    start_col (str): Nome da coluna com o tempo inicial.
    end_col (str): Nome da coluna com o tempo final.

    Retorna:
    pd.Timedelta: Diferença de tempo entre 'start_col' e 'end_col'.
    """
    # Extrai os tempos iniciais e finais das colunas especificadas.
    start_time = row[start_col]
    end_time = row[end_col]
    
    # Verifica se algum dos tempos é NaT (Not a Time) e retorna 0 se verdadeiro.
    if pd.isna(start_time) or pd.isna(end_time):
        return 0
    
    # Calcula e retorna a diferença absoluta de tempo entre os dois tempos.
    delta = abs(end_time - start_time)
    return delta

def calculate_time_int(row, start_col, end_col):
    """
    Calcula a diferença de tempo em minutos entre duas colunas de um DataFrame.

    Parâmetros:
    row (pd.Series): Uma linha do DataFrame.
    start_col (str): Nome da coluna com o tempo inicial.
    end_col (str): Nome da coluna com o tempo final.

    Retorna:
    int: Diferença de tempo em minutos entre 'start_col' e 'end_col'.
    """
    # Extrai os tempos iniciais e finais das colunas especificadas.
    start_time = row[start_col]
    end_time = row[end_col]
    
    # Verifica se algum dos tempos é NaT (Not a Time) e retorna 0 se verdadeiro.
    if pd.isna(start_time) or pd.isna(end_time):
        return 0
    
    # Calcula a diferença de tempo, converte para segundos e depois para minutos.
    delta = abs(end_time - start_time)
    return int(delta.total_seconds() / 60)

def encontrar_coluna_similar(coluna, colunas_padrao, limiar=8):
    """
    Encontra a coluna mais similar a um dado nome de coluna, baseado em uma medida de distância.

    Args:
    coluna (str): Nome da coluna a ser comparada.
    colunas_padrao (list): Lista de nomes de colunas padrão para comparação.
    limiar (int, optional): Limiar de distância para considerar uma coluna similar. Default é 8.

    Etapas:
    1. Iterar sobre cada coluna padrão.
    2. Calcular a distância de Levenshtein entre a coluna e cada coluna padrão.
    3. Se a distância for menor ou igual ao limiar, retornar a coluna padrão correspondente.
    4. Se nenhuma coluna padrão satisfizer o critério, retornar o nome da coluna original.

    Returns:
    str: O nome da coluna mais similar dentre as colunas padrão, ou o próprio nome da coluna se nenhuma for suficientemente similar.
    """
     # Itera sobre cada coluna padrão na lista fornecida.
    for padrao in colunas_padrao:
        # Calcula a distância de Levenshtein entre a coluna e a coluna padrão atual.
        # A função lev.distance() é assumida como uma função predefinida ou importada para calcular esta distância.
        if lev.distance(coluna, padrao) <= limiar:
            # Se a distância for menor ou igual ao limiar, retorna a coluna padrão como correspondente.
            return padrao
    # Se nenhuma coluna padrão corresponder dentro do limiar, retorna o nome da coluna original.
    return coluna

# Função para detectar o delimitador mais adequado
def detectar_delimitador(filename, encoding='utf-8', num_lines=10):
    """
    Detecta o delimitador mais adequado para um arquivo CSV.

    Args:
    filename (str): Nome do arquivo a ser analisado.
    encoding (str, optional): Codificação do arquivo. Default é 'utf-8'.
    num_lines (int, optional): Número de linhas a serem lidas para a detecção do delimitador. Default é 10.

    Etapas:
    1. Definir uma lista de delimitadores possíveis.
    2. Ler as primeiras linhas do arquivo, testando cada delimitador possível.
    3. Identificar o delimitador que resulta no maior número de campos (colunas) por linha.
    4. Retornar o delimitador que maximiza o número de campos.

    Returns:
    str: O delimitador mais adequado encontrado no arquivo. Se nenhum delimitador adequado for encontrado, retorna None.
    """
    # Define uma lista de delimitadores possíveis para testar no arquivo.
    delimitadores_possiveis = [',', ';', '\t', '|']
    # Inicializa variáveis para armazenar o delimitador com o máximo de campos e o número máximo de campos.
    max_delimiter = None
    max_fields = 1

    # Abre o arquivo com o nome e codificação especificados.
    with open(filename, 'r', encoding=encoding) as file:
        # Testa cada delimitador possível.
        for delimiter in delimitadores_possiveis:
            # Retorna ao início do arquivo antes de testar um novo delimitador.
            file.seek(0)
            # Cria um leitor CSV com o delimitador atual.
            reader = csv.reader(file, delimiter=delimiter)
            # Itera sobre as linhas do arquivo até o limite especificado.
            for i, row in enumerate(reader):
                # Interrompe a leitura após um determinado número de linhas.
                if i >= num_lines:
                    break
                # Atualiza o delimitador e o número máximo de campos se este delimitador resultar em mais campos.
                if len(row) > max_fields:
                    max_fields = len(row)
                    max_delimiter = delimiter

    # Retorna o delimitador que resultou no maior número de campos por linha.
    return max_delimiter

# Carregando dados

In [3]:
# --- Carregamento dos Dados dos Aeroportos ---
# Definindo o caminho para o arquivo CSV dos glossários de aeródromos.
file_path = "dados_complementares_convertidos/glossario_de_aerodromo.csv"

# Detectando a codificação do arquivo de aeródromos para assegurar a leitura correta dos dados.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para o arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# A codificação detectada é usada para lidar com possíveis caracteres especiais nos dados.
df_aeroportos = pd.read_csv(file_path, sep=',', encoding=encoding)  # A codificação pode variar (ex: 'iso-8859-1', 'cp1252'), dependendo do arquivo.


# --- Carregamento dos Dados das Companhias Aéreas ---
# Definindo o caminho para o arquivo CSV dos glossários de empresas aéreas.
file_path = "dados_complementares_convertidos/glossario_de_empresas_aereas.csv"

# Detectando a codificação do arquivo das empresas aéreas.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para este arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# Novamente, a codificação detectada é essencial para lidar com caracteres especiais.
df_cia_aerea = pd.read_csv(file_path, sep=',', encoding=encoding)  # A codificação pode variar, semelhante ao arquivo de aeródromos.

# --- Carregamento dos Dados das Companhias Aéreas ---
# Definindo o caminho para o arquivo CSV com algumas informações de aeroportos.
file_path = "dados_tratados/dados_aeroportos_tratado.csv"

# Detectando a codificação do arquivo.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para este arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# Novamente, a codificação detectada é essencial para lidar com caracteres especiais.
df_aeroportos_2 = pd.read_csv(file_path, sep=',', encoding=encoding)  # A codificação pode variar, semelhante ao arquivo de aeródromos.

In [4]:
mapeamento_colunas = {
    'Código Autorização (DI)': 'Codigo DI',
    'cd_di': 'Codigo DI',
    'ICAO Empresa Aerea': 'ICAO Empresa Aérea',
    'sg_empresa_icao': 'ICAO Empresa Aérea',
    'Número Voo': 'Número Voo',
    'NUmero Voo': 'Número Voo',
    'Numero Voo': 'Número Voo',
    'nr_voo': 'Número Voo',
    'Código Tipo Linha': 'Código Tipo Linha',
    'Codigo Tipo Linha': 'Código Tipo Linha',
    'cd_tipo_linha': 'Código Tipo Linha',
    'ICAO Aeródromo Origem': 'ICAO Aeródromo Origem',
    'ICAO Aerodromo Origem': 'ICAO Aeródromo Origem',
    'sg_icao_origem': 'ICAO Aeródromo Origem',
    'ICAO Aeródromo Destino': 'ICAO Aeródromo Destino',
    'ICAO Aerodromo Destino': 'ICAO Aeródromo Destino',
    'sg_icao_destino': 'ICAO Aeródromo Destino',
    'Partida Prevista': 'Partida Prevista',
    'dt_partida_prevista': 'Partida Prevista',
    'Partida Real': 'Partida Real',
    'Código DI': 'Codigo DI',   
    'Data Partida Real': 'Partida Real',
    'Data Chegada Real': 'Chegada Real',
    'Data Partida Prevista': 'Partida Prevista',
    'Data Chegada Prevista': 'Chegada Prevista',
    'dt_partida_real': 'Partida Real',
    'Chegada Prevista': 'Chegada Prevista',
    'dt_chegada_prevista': 'Chegada Prevista',
    'Chegada Real': 'Chegada Real',
    'dt_chegada_real': 'Chegada Real',
    'Situação Voo': 'Situação Voo',
    'Situacao Voo': 'Situação Voo',
    'situacao': 'Situação Voo',
    'Código Justificativa': 'Código Justificativa',
    'Codigo Justificativa': 'Código Justificativa',
    'cd_justificativa': 'Código Justificativa',
    'ICAO Aer—dromo Origem': 'ICAO Aeródromo Origem',
    'ICAO Aer—dromo Destino': 'ICAO Aeródromo Destino',
    'Sigla ICAO Aeroporto Origem': 'ICAO Aeródromo Origem',
    'Sigla ICAO Aeroporto Destino': 'ICAO Aeródromo Destino',
    'Sigla ICAO Empresa Aérea': 'ICAO Empresa Aérea',
}

# Colunas padrão esperadas
colunas_esperadas = [
    'ICAO Empresa Aérea', 'Número Voo', 'Código Tipo Linha',
    'ICAO Aeródromo Origem', 'ICAO Aeródromo Destino', 'Partida Prevista', 'Partida Real',
    'Chegada Prevista', 'Chegada Real', 'Situação Voo', 'Código Justificativa', 'Codigo DI'
]

# Definindo o caminho para os arquivos CSV do ano de 2019.
file_path = "2023/*.csv"

# Detectando a codificação dos arquivos CSV
file_encodings = detect_encoding(file_path)
codificacoes_comuns = ['utf-8', 'latin1', 'ISO-8859-1', 'ascii','utf-32','utf-16']
# Processamento dos arquivos CSV
df_list = []
for filename in glob.glob(file_path):
    print(f"Carregando arquivo: {filename}")
    for encoding in codificacoes_comuns:
        try:
            delimitador = detectar_delimitador(filename, encoding=encoding)
            df = pd.read_csv(filename, encoding=encoding, sep=delimitador, low_memory=False, index_col=None)

            # Renomear colunas com base no mapeamento
            df.rename(columns=mapeamento_colunas, inplace=True, errors='ignore')

            # Manter apenas as colunas esperadas que existem no DataFrame
            colunas_existentes = [col for col in colunas_esperadas if col in df.columns]
            df = df[colunas_existentes]

            df_list.append(df)
            print(f"Arquivo {filename} carregado com sucesso usando a codificação {encoding}.")
            break
        except UnicodeDecodeError as e:
            print(f"Erro de decodificação com a codificação {encoding} no arquivo {filename}: {e}")
        except Exception as e:
            print(f"Erro ao carregar o arquivo {filename}: {e}")
            break

# Concatenando todos os DataFrames em um único DataFrame para análise combinada.
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print("Todos os arquivos CSV foram carregados com sucesso.")
else:
    print("Nenhum arquivo CSV foi encontrado ou carregado.")

Carregando arquivo: 2023\vra-01_2023.csv
Arquivo 2023\vra-01_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-02_2023.csv
Arquivo 2023\vra-02_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-03_2023.csv
Arquivo 2023\vra-03_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-04_2023.csv
Arquivo 2023\vra-04_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-05_2023.csv
Arquivo 2023\vra-05_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-06_2023.csv
Arquivo 2023\vra-06_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-07_2023.csv
Arquivo 2023\vra-07_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-08_2023.csv
Arquivo 2023\vra-08_2023.csv carregado com sucesso usando a codificação utf-8.
Carregando arquivo: 2023\vra-09_2023.csv

In [5]:
df = combined_df.copy()

## Dicionário de dados
| Campo                         | Descrição                                                                                                   |
|-------------------------------|-------------------------------------------------------------------------------------------------------------|
| Sigla ICAO Empresa Aérea      | Sigla/Designador ICAO Empresa Aérea                                                                         |
| Número Voo                    | Numeração do voo                                                                                            |
| Código DI                     | Caractere usado para identificar o Dígito Identificador (DI) para cada etapa de voo                         |
| Código Tipo Linha             | Caractere usado para identificar o Tipo de Linha realizada para cada etapa de voo                           |
| Sigla ICAO Aeroporto Origem   | Sigla/Designador ICAO Aeroporto de Origem                                                                   |
| Sigla ICAO Aeroporto Destino  | Sigla/Designador ICAO Aeroporto de Destino                                                                  |
| Partida Prevista              | Data e horário da partida prevista informada pela empresa aérea, em horário de Brasília                      |
| Partida Real                  | Data e horário da partida realizada informada pela empresa aérea, em horário de Brasília                     |
| Chegada Prevista              | Data e horário da chegada prevista informada pela empresa aérea, em horário de Brasília                      |
| Chegada Real                  | Data e horário da chegada realizada, informada pela empresa aérea, em horário de Brasília                    |
| Situação do voo               | Campo informando a situação do voo: realizado, cancelado ou não informado.                                  |


# Tratamento de dados

## Join com as tabelas complementares

In [6]:
# Preparando DataFrames de Aeroportos para Mesclagem com o DataFrame Principal.

# Criando cópias do DataFrame de aeroportos para uso separado como origem e destino.
df_aeroportos_origem = df_aeroportos.copy()
df_aeroportos_destino = df_aeroportos.copy()

# Renomeando as colunas do DataFrame de aeroportos origem.
# Isso é feito para evitar conflitos de nomes ao mesclar com o DataFrame principal.
# Cada nome de coluna é acrescido de '_origem' para indicar que se refere ao aeroporto de origem do voo.
df_aeroportos_origem.rename(columns={col: col + '_origem' for col in df_aeroportos_origem.columns}, inplace=True)

# Renomeando as colunas do DataFrame de aeroportos destino de forma similar.
# Aqui, '_destino' é acrescido para indicar que as colunas se referem ao aeroporto de destino.
df_aeroportos_destino.rename(columns={col: col + '_destino' for col in df_aeroportos_destino.columns}, inplace=True)

# Mesclando o DataFrame principal com o DataFrame de companhias aéreas.
# A mesclagem é feita com base na coluna 'ICAO Empresa Aérea' do DataFrame principal
# e na coluna 'Sigla OACI' do DataFrame das companhias aéreas.
# A opção 'inner' assegura que apenas os registros presentes em ambos os DataFrames sejam mantidos.
df = pd.merge(df, df_cia_aerea, left_on='ICAO Empresa Aérea', right_on='Sigla OACI', how='inner')

# Mesclando o DataFrame resultante com o DataFrame de aeroportos origem.
# A mesclagem utiliza 'ICAO Aeródromo Origem' do DataFrame principal e 'Sigla OACI_origem' do DataFrame de aeroportos origem.
df = pd.merge(df, df_aeroportos_origem, left_on='ICAO Aeródromo Origem', right_on='Sigla OACI_origem', how='inner')

# Mesclando o DataFrame resultante com o DataFrame de aeroportos destino.
# Similar à etapa anterior, mas agora usando 'ICAO Aeródromo Destino' e 'Sigla OACI_destino'.
df = pd.merge(df, df_aeroportos_destino, left_on='ICAO Aeródromo Destino', right_on='Sigla OACI_destino', how='inner')


## Filtro de cia aerea 

In [7]:
# Filtrando o DataFrame para incluir apenas voos das empresas aéreas específicas.

# Definindo a lista de códigos ICAO das empresas aéreas desejadas para o filtro.
valores_filtrar = ['GLO', 'AZU', 'TAM']

# Filtrando o DataFrame para manter apenas as linhas onde 'ICAO Empresa Aérea' corresponde aos valores na lista de filtro.
df = df[df['ICAO Empresa Aérea'].isin(valores_filtrar)]

## Datas

In [8]:
# Convertendo colunas de datas e horas de partidas e chegadas para o formato datetime.

# As seguintes colunas são convertidas para o tipo datetime usando o formato especificado '%d/%m/%Y %H:%M':
# 'Partida Prevista', 'Partida Real', 'Chegada Prevista' e 'Chegada Real'.
df['Partida Prevista'] = pd.to_datetime(df['Partida Prevista'], format='%d/%m/%Y %H:%M')
df['Partida Real'] = pd.to_datetime(df['Partida Real'], format='%d/%m/%Y %H:%M')
df['Chegada Prevista'] = pd.to_datetime(df['Chegada Prevista'], format='%d/%m/%Y %H:%M')
df['Chegada Real'] = pd.to_datetime(df['Chegada Real'], format='%d/%m/%Y %H:%M')

# Aplicando a função 'is_delayed' em cada linha para determinar se o voo está atrasado ou pontual.
# O resultado é armazenado na nova coluna 'Status do Voo'.
df['Status do Voo'] = df.apply(is_delayed, axis=1)

### Criação de colunas com delta

In [9]:
# Aplicando a função para calcular o delta de tempo
df['Delta Tempo Partida delta'] = df.apply(calculate_time_delta, axis=1, start_col='Partida Prevista', end_col='Partida Real')
df['Delta Tempo Chegada delta'] = df.apply(calculate_time_delta, axis=1, start_col='Chegada Prevista', end_col='Chegada Real')

# Aplicando a função para calcular o delta de tempo
df['Delta Tempo Partida int'] = df.apply(calculate_time_int, axis=1, start_col='Partida Prevista', end_col='Partida Real')
df['Delta Tempo Chegada int'] = df.apply(calculate_time_int, axis=1, start_col='Chegada Prevista', end_col='Chegada Real')

## Seleção de colunas

In [10]:
df

,ICAO Empresa Aérea,Número Voo,Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Codigo DI,Sigla OACI,Nome Empresas,Nacional ou Estrangeira,Sigla OACI_origem,Descrição_origem,Cidade_origem,UF_origem,País_origem,Continente_origem,Sigla OACI_destino,Descrição_destino,Cidade_destino,UF_destino,País_destino,Continente_destino,Status do Voo,Delta Tempo Partida delta,Delta Tempo Chegada delta,Delta Tempo Partida int,Delta Tempo Chegada int
402,TAM,8191,I,KMIA,KMIA,NaT,2023-03-16 21:15:00,NaT,2023-03-16 21:40:00,REALIZADO,3,TAM,TAM,BRASILEIRA,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0
403,TAM,8195,I,KMIA,KMIA,NaT,2023-03-28 11:33:00,NaT,2023-03-28 12:04:00,REALIZADO,3,TAM,TAM,BRASILEIRA,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0
404,TAM,8191,I,KMIA,KMIA,NaT,2023-06-11 21:36:00,NaT,2023-06-11 21:50:00,REALIZADO,3,TAM,TAM,BRASILEIRA,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0
405,TAM,8126,I,KMIA,KMIA,NaT,2023-09-11 23:58:00,NaT,2023-09-12 00:15:00,REALIZADO,3,TAM,TAM,BRASILEIRA,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0
406,TAM,8195,I,KMIA,KMIA,NaT,2023-11-11 11:24:00,NaT,2023-11-11 11:45:00,REALIZADO,3,TAM,TAM,BRASILEIRA,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,KMIA,MIAMI INTERNATIONAL AIRPORT,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919254,GLO,9675,I,GVAC,LBSF,NaT,2023-04-15 08:44:00,NaT,2023-04-15 15:30:00,REALIZADO,6,GLO,GOL,BRASILEIRA,GVAC,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,ILHA DO SAL,NaN,CABO VERDE,AFRICA,LBSF,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),SOFIA,NaN,BULGARIA,EUROPA,Pontual,0,0,0,0
919255,GLO,9678,I,GVAC,LBSF,NaT,2023-08-20 05:42:00,NaT,2023-08-20 12:50:00,REALIZADO,6,GLO,GOL,BRASILEIRA,GVAC,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,ILHA DO SAL,NaN,CABO VERDE,AFRICA,LBSF,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),SOFIA,NaN,BULGARIA,EUROPA,Pontual,0,0,0,0
919256,GLO,9678,I,GVAC,LBSF,NaT,2023-09-12 05:00:00,NaT,2023-09-12 11:44:00,REALIZADO,6,GLO,GOL,BRASILEIRA,GVAC,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,ILHA DO SAL,NaN,CABO VERDE,AFRICA,LBSF,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),SOFIA,NaN,BULGARIA,EUROPA,Pontual,0,0,0,0
919257,GLO,9678,I,OMDW,LBSF,NaT,2023-10-01 05:12:00,NaT,2023-10-01 10:51:00,REALIZADO,6,GLO,GOL,BRASILEIRA,OMDW,DUBAI INTERNATIONAL AIRPORT,NaN,NaN,EMIRADOSARABES UNIDOS,ASIA,LBSF,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),SOFIA,NaN,BULGARIA,EUROPA,Pontual,0,0,0,0


In [11]:
df= df[['Nome Empresas','Número Voo', 'Codigo DI','ICAO Aeródromo Origem', 'ICAO Aeródromo Destino',
 'Código Tipo Linha', 'Partida Prevista', 'Partida Real',
 'Chegada Prevista', 'Chegada Real', 'Situação Voo',
 'Descrição_origem', 'País_origem', 'Continente_origem',
 'Descrição_destino', 'País_destino', 'Continente_destino',
 'Status do Voo', 'Delta Tempo Partida delta', 'Delta Tempo Chegada delta',
 'Delta Tempo Partida int', 'Delta Tempo Chegada int', 'Cidade_origem',
 'UF_origem', 'Cidade_destino', 'UF_destino']]

## Removendo caracteres especiais e espaço do nome das colunas

In [12]:
# Aplicar a função a cada nome de coluna
df.columns = [padronizar_nome_coluna(col) for col in df.columns]

# Adiação de colunas mês, dia do mês e dia da semana

In [13]:
# Extração de componentes de data das colunas de partida e chegada.

# As seguintes operações são realizadas para extrair informações específicas das datas de partida:
# 1. 'mes_partida': Extração do mês da 'partida_prevista'.
# 2. 'dia_semana_partida': Extração do dia da semana da 'partida_prevista' (0 = Segunda-feira, 6 = Domingo).
# 3. 'dia_mes_partida': Extração do dia do mês da 'partida_prevista'.
df['mes_partida'] = df['partida_prevista'].dt.month
df['dia_semana_partida'] = df['partida_prevista'].dt.dayofweek
df['dia_mes_partida'] = df['partida_prevista'].dt.day
df['hora_partida'] = df['partida_prevista'].dt.hour
# Nota: As mesmas informações são extraídas para 'chegada_prevista', mas há um erro no código;
# 'partida_prevista' está sendo usada ao invés de 'chegada_prevista'.
# As variáveis 'mes_chegada', 'dia_semana_chegada', e 'dia_mes_chegada' devem ser ajustadas para usar 'chegada_prevista'.
df['mes_chegada'] = df['chegada_prevista'].dt.month
df['dia_semana_chegada'] = df['chegada_prevista'].dt.dayofweek
df['dia_mes_chegada'] = df['chegada_prevista'].dt.day
df['hora_chegada'] = df['chegada_prevista'].dt.hour


C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\3335256749.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mes_partida'] = df['partida_prevista'].dt.month
C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\3335256749.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dia_semana_partida'] = df['partida_prevista'].dt.dayofweek
C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\3335256749.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

# Adicionando informações adicionais do Aeroporto

In [14]:
# Preparando DataFrames de Aeroportos para Mesclagem com o DataFrame Principal.

# Criando cópias do DataFrame de aeroportos para uso separado como origem e destino.
df_aeroportos_origem = df_aeroportos_2.copy()
df_aeroportos_destino = df_aeroportos_2.copy()

# Renomeando as colunas do DataFrame de aeroportos origem.
# Isso é feito para evitar conflitos de nomes ao mesclar com o DataFrame principal.
# Cada nome de coluna é acrescido de '_origem' para indicar que se refere ao aeroporto de origem do voo.
df_aeroportos_origem.rename(columns={col: col + '_origem' for col in df_aeroportos_origem.columns}, inplace=True)

# Renomeando as colunas do DataFrame de aeroportos destino de forma similar.
# Aqui, '_destino' é acrescido para indicar que as colunas se referem ao aeroporto de destino.
df_aeroportos_destino.rename(columns={col: col + '_destino' for col in df_aeroportos_destino.columns}, inplace=True)

# Mesclando o DataFrame resultante com o DataFrame de aeroportos origem.
# A mesclagem utiliza 'ICAO Aeródromo Origem' do DataFrame principal e 'Sigla OACI_origem' do DataFrame de aeroportos origem.
df = pd.merge(df, df_aeroportos_origem, left_on='icao_aerodromo_origem', right_on='codigo_oaci_origem', how='left')

# Mesclando o DataFrame resultante com o DataFrame de aeroportos destino.
# Similar à etapa anterior, mas agora usando 'ICAO Aeródromo Destino' e 'Sigla OACI_destino'.
df = pd.merge(df, df_aeroportos_destino, left_on='icao_aerodromo_destino', right_on='codigo_oaci_destino', how='left')

In [15]:
df

,nome_empresas,numero_voo,codigo_di,icao_aerodromo_origem,icao_aerodromo_destino,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,cidade_origem,uf_origem_x,cidade_destino,uf_destino_x,mes_partida,dia_semana_partida,dia_mes_partida,hora_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,hora_chegada,codigo_oaci_origem,ciad_origem,nome_origem,municipio_origem,uf_origem_y,altitude_origem,latgeopoint_origem,longeopoint_origem,latitude_aero_origem,longitude_aero_origem,codigo_oaci_destino,ciad_destino,nome_destino,municipio_destino,uf_destino_y,altitude_destino,latgeopoint_destino,longeopoint_destino,latitude_aero_destino,longitude_aero_destino
0,TAM,8191,3,KMIA,KMIA,I,NaT,2023-03-16 21:15:00,NaT,2023-03-16 21:40:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TAM,8195,3,KMIA,KMIA,I,NaT,2023-03-28 11:33:00,NaT,2023-03-28 12:04:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TAM,8191,3,KMIA,KMIA,I,NaT,2023-06-11 21:36:00,NaT,2023-06-11 21:50:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TAM,8126,3,KMIA,KMIA,I,NaT,2023-09-11 23:58:00,NaT,2023-09-12 00:15:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TAM,8195,3,KMIA,KMIA,I,NaT,2023-11-11 11:24:00,NaT,2023-11-11 11:45:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788619,GOL,9675,6,GVAC,LBSF,I,NaT,2023-04-15 08:44:00,NaT,2023-04-15 15:30:00,REALIZADO,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
788620,GOL,9678,6,GVAC,LBSF,I,NaT,2023-08-20 05:42:00,NaT,2023-08-20 12:50:00,REALIZADO,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
788621,GOL,9678,6,GVAC,LBSF,I,NaT,2023-09-12 05:00:00,NaT,2023-09-12 11:44:00,REALIZADO,AEROPORTO INTERNACIONAL AMÃLCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT (VRAZHDEBNA),BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Padronização e seleção de colunas

In [16]:
df =  df[['nome_empresas', 'numero_voo', 'codigo_di', 'codigo_tipo_linha',
 'partida_prevista', 'partida_real', 'chegada_prevista',
 'chegada_real', 'situacao_voo', 'descricao_origem', 'pais_origem',
 'continente_origem', 'descricao_destino', 'pais_destino',
 'continente_destino', 'status_do_voo', 'delta_tempo_partida_delta',
 'delta_tempo_chegada_delta', 'delta_tempo_partida_int', 'delta_tempo_chegada_int',
 'cidade_origem', 'uf_origem_x', 'cidade_destino', 'uf_destino_x', 'mes_partida',
 'dia_semana_partida', 'dia_mes_partida', 'hora_partida',
 'mes_chegada', 'dia_semana_chegada', 'dia_mes_chegada',
 'hora_chegada',  'altitude_origem', 'latgeopoint_origem',
 'longeopoint_origem', 'latitude_aero_origem', 'longitude_aero_origem',
 'altitude_destino', 'latgeopoint_destino', 'longeopoint_destino',
 'latitude_aero_destino', 'longitude_aero_destino']]

In [17]:
def clean_name(name):
    # Remove content inside parentheses
    name = pd.Series(name).replace(r'\(.*?\)', '', regex=True).str.strip()

    # Remove accents and special characters
    return name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

In [18]:
# Applying the cleaning function to the 'uf' column
df['cidade_origem'] = clean_name(df['cidade_origem'])
df['cidade_destino'] = clean_name(df['cidade_destino'])
df['descricao_origem'] = clean_name(df['descricao_origem'])
df['descricao_destino'] = clean_name(df['descricao_destino'])

C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\2003219343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cidade_origem'] = clean_name(df['cidade_origem'])
C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\2003219343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cidade_destino'] = clean_name(df['cidade_destino'])
C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\2003219343.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

# Criando coluna com as principais rotas

In [19]:
# Criação da variavel rota (aeroporto origem + aeroporto destino). Observar as rotas mais problemas de atraso e cancelamento
df['rota'] = df['descricao_origem'] + " -> " + df['descricao_destino']

C:\Users\pedro\AppData\Local\Temp\ipykernel_15524\1053866737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rota'] = df['descricao_origem'] + " -> " + df['descricao_destino']


# Salvando os dados tratatos para realizar o join com dados meteorologicos

In [21]:
# Definir o caminho do diretório e do arquivo CSV
diretorio = 'dados_tratados'
nome_arquivo = 'historico_voo_tratados_2023.csv'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Verificar se o diretório existe. Se não, criar o diretório
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Salvar o DataFrame no arquivo CSV
df.to_csv(caminho_completo, index=False)

In [20]:
df

,nome_empresas,numero_voo,codigo_di,codigo_tipo_linha,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,descricao_origem,pais_origem,continente_origem,descricao_destino,pais_destino,continente_destino,status_do_voo,delta_tempo_partida_delta,delta_tempo_chegada_delta,delta_tempo_partida_int,delta_tempo_chegada_int,cidade_origem,uf_origem_x,cidade_destino,uf_destino_x,mes_partida,dia_semana_partida,dia_mes_partida,hora_partida,mes_chegada,dia_semana_chegada,dia_mes_chegada,hora_chegada,altitude_origem,latgeopoint_origem,longeopoint_origem,latitude_aero_origem,longitude_aero_origem,altitude_destino,latgeopoint_destino,longeopoint_destino,latitude_aero_destino,longitude_aero_destino,rota
0,TAM,8191,3,I,NaT,2023-03-16 21:15:00,NaT,2023-03-16 21:40:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIAMI INTERNATIONAL AIRPORT -> MIAMI INTERNATI...
1,TAM,8195,3,I,NaT,2023-03-28 11:33:00,NaT,2023-03-28 12:04:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIAMI INTERNATIONAL AIRPORT -> MIAMI INTERNATI...
2,TAM,8191,3,I,NaT,2023-06-11 21:36:00,NaT,2023-06-11 21:50:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIAMI INTERNATIONAL AIRPORT -> MIAMI INTERNATI...
3,TAM,8126,3,I,NaT,2023-09-11 23:58:00,NaT,2023-09-12 00:15:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIAMI INTERNATIONAL AIRPORT -> MIAMI INTERNATI...
4,TAM,8195,3,I,NaT,2023-11-11 11:24:00,NaT,2023-11-11 11:45:00,REALIZADO,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,MIAMI INTERNATIONAL AIRPORT,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,Pontual,0,0,0,0,MIAMI,NaN,MIAMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIAMI INTERNATIONAL AIRPORT -> MIAMI INTERNATI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788619,GOL,9675,6,I,NaT,2023-04-15 08:44:00,NaT,2023-04-15 15:30:00,REALIZADO,AEROPORTO INTERNACIONAL AMALCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT,BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEROPORTO INTERNACIONAL AMALCAR CABRAL -> SOFI...
788620,GOL,9678,6,I,NaT,2023-08-20 05:42:00,NaT,2023-08-20 12:50:00,REALIZADO,AEROPORTO INTERNACIONAL AMALCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT,BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEROPORTO INTERNACIONAL AMALCAR CABRAL -> SOFI...
788621,GOL,9678,6,I,NaT,2023-09-12 05:00:00,NaT,2023-09-12 11:44:00,REALIZADO,AEROPORTO INTERNACIONAL AMALCAR CABRAL,CABO VERDE,AFRICA,SOFIA INTERNATIONAL AIRPORT,BULGARIA,EUROPA,Pontual,0,0,0,0,ILHA DO SAL,NaN,SOFIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AEROPORTO INTERNACIONAL AMALCAR CABRAL -> SOFI...
788622,GOL,9678,6,I,NaT,2023-10-01 05:12:00,NaT,2023-10-01 10:51:00,REALIZADO,DUBAI INTERNATIONAL AIRPORT,EMIRADOSARABES UNIDOS,ASIA,SOFIA INTERNATIONAL AIRPORT,BULGARIA,EUROPA,Pontual,0,0,0,0,NaN,NaN,SOFIA,NaN,Na

In [ ]:
# Definir o caminho do diretório e do arquivo CSV
diretorio = 'dados_tratados'
nome_arquivo = 'historico_voo_tratados_2019.parquet'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Verificar se o diretório existe. Se não, criar o diretório
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Salvar o DataFrame no arquivo CSV
df.to_parquet(caminho_completo, index=False)

In [116]:
diretorio = 'dados_tratados'
nome_arquivo = 'historico_voo_tratados_2019.parquet'
caminho_completo = os.path.join(diretorio, nome_arquivo)
df.to_parquet(caminho_completo)